In [1]:
import numpy as np
import statsmodels.api as sm

# Datos de entrada
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])

# Agregar constante (para incluir la ordenada al origen)
X = sm.add_constant(x)  # Esto agrega una columna de unos para el término constante

# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Mostrar los resultados
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     16.35
Date:                Sun, 08 Dec 2024   Prob (F-statistic):             0.0272
Time:                        17:24:46   Log-Likelihood:                -21.674
No. Observations:                   5   AIC:                             47.35
Df Residuals:                       3   BIC:                             46.57
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -4.441e-15     10.661  -4.17e-16      1.0

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [21]:
import numpy as np

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])

# Varianza del error
sigma_squared = 100

# Calcular x^T x
xtx = np.sum(x**2)

# Calcular beta sombrero
beta_hat = np.sum(x * y) / xtx

# Calcular la varianza de beta sombrero
var_beta_hat = sigma_squared / xtx

# Resultados
{
    "beta_hat": beta_hat,
    "var_beta_hat": var_beta_hat
}


{'beta_hat': 6.5, 'var_beta_hat': 0.45454545454545453}

In [24]:
y_hat = beta_hat * x 
e_hat = y - y_hat
n = len(x)
k = 1
sigma_hat = np.sum(e_hat**2) / (n-k)
sigma_hat

426.25

In [26]:
y_hat

array([ 19.5,  45.5, -26. ,  32.5, -71.5])

In [25]:
sigma_hat / xtx

1.9375

In [20]:
np.sum(x * y)

1430

In [16]:
import numpy as np

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])
z = np.array([1, 2, -2, 4, -5])

# Varianza del error
sigma_squared = 100

# Paso 1: Regresión de x sobre z
ztz = np.sum(z**2)  # z^T z
ztx = np.sum(z * x)  # z^T x
pi_hat = ztx / ztz   # Coeficiente estimado

# Calcular x sombrero
x_hat = pi_hat * z

# Paso 2: Regresión de y sobre x sombrero
x_hat_t_x_hat = np.sum(x_hat**2)  # (x sombrero)^T x sombrero
x_hat_t_y = np.sum(x_hat * y)     # (x sombrero)^T y
beta_hat_2sls = x_hat_t_y / x_hat_t_x_hat

# Calcular la varianza de beta sombrero
var_beta_hat_2sls = sigma_squared / x_hat_t_x_hat

# Resultados
{
    "pi_hat (coeficiente x sobre z)": pi_hat,
    "x_hat": x_hat,
    "beta_hat_2sls": beta_hat_2sls,
    "var_beta_hat_2sls": var_beta_hat_2sls
}


{'pi_hat (coeficiente x sobre z)': 2.0,
 'x_hat': array([  2.,   4.,  -4.,   8., -10.]),
 'beta_hat_2sls': 7.2,
 'var_beta_hat_2sls': 0.5}

In [17]:
var_beta_hat = 0.45454545454545453
var_beta_hat_2sls = 0.5

(6.5 - 7.2) / np.sqrt(var_beta_hat_2sls - var_beta_hat)

-3.283291031876401

In [37]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import chi2

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])
z = np.array([1, 2, -2, 4, -5])

# Paso 1: Estimación por OLS sin constante
model_ols = sm.OLS(y, x).fit()  # OLS sin constante
beta_ols = model_ols.params
var_beta_ols = 0.45454545

# Paso 2: Estimación por 2SLS sin constante
# Etapa 1: Regresión de x sobre z (sin constante)
model_stage1 = sm.OLS(x, z).fit()
x_hat = model_stage1.predict(z)

# Etapa 2: Regresión de y sobre x sombrero (sin constante)
model_2sls = sm.OLS(y, x_hat).fit()
beta_2sls = model_2sls.params
var_beta_2sls = 0.5

# Paso 3: Estadístico de Hausman
beta_diff = beta_ols - beta_2sls  # Diferencia entre los estimadores
var_diff =  var_beta_2sls - var_beta_ols  # Diferencia entre varianzas
hausman_stat = beta_diff.T * var_diff * beta_diff  # Estadístico

# Usar scipy.stats.chi2.sf para calcular el p-valor
p_value = 1 - chi2.sf(hausman_stat, df=1)  # p-valor, 1 parámetro

# Resultados
{
    "beta_ols": beta_ols,
    "beta_2sls": beta_2sls,
    "hausman_stat": hausman_stat,
    "p_value": p_value
}



{'beta_ols': array([6.5]),
 'beta_2sls': array([7.2]),
 'hausman_stat': array([0.02227273]),
 'p_value': array([0.11863614])}

In [47]:
from scipy.stats import chi2

# Grados de libertad
df = 2  # Cambia según el número de parámetros que estás probando

# Punto crítico al 5% en la cola derecha
chi2_critical = chi2.ppf(0.95, df)

print(f"Punto crítico al 5% (chi-cuadrado, df={df}): {chi2_critical}")


Punto crítico al 5% (chi-cuadrado, df=2): 5.991464547107979


In [14]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm

# Datos
x = np.array([3, 7, -4, 5, -11])
y = np.array([20, 30, -50, 60, -60])
z = np.array([1, 2, -2, 4, -5])

# Estimación por OLS sin constante (sin agregar constante)
model_ols = sm.OLS(y, x).fit()  # OLS sin constante
beta_ols = model_ols.params[0]  # Coeficiente estimado por OLS
var_beta_ols = model_ols.cov_params()[0, 0]  # Varianza del estimador OLS

# Estimación por 2SLS sin constante
# Etapa 1: Regresión de x sobre z (sin constante)
model_stage1 = sm.OLS(x, z).fit()
x_hat = model_stage1.predict(z)

# Etapa 2: Regresión de y sobre x sombrero (sin constante)
model_2sls = sm.OLS(y, x_hat).fit()
beta_2sls = model_2sls.params[0]  # Coeficiente estimado por 2SLS
var_beta_2sls = model_2sls.cov_params()[0, 0]  # Varianza del estimador 2SLS

# Estimación por MCC
# Para ilustrar, tomamos los mismos resultados de OLS como MCC (esto es solo un ejemplo)
beta_mcc = beta_ols
var_beta_mcc = var_beta_ols

# Calcular el estadístico de Hausman
beta_diff = beta_ols - beta_2sls  # Diferencia entre los estimadores
var_diff = var_beta_2sls - var_beta_mcc  # Diferencia entre las varianzas (2SLS - MCC)

# Estadístico de Hausman
hausman_stat = beta_diff / np.sqrt(var_diff)

# Calcular el p-valor usando la distribución normal estándar (asumiendo que sigue N(0,1))
p_value = 1 - norm.cdf(hausman_stat)  # p-valor

# Resultados
{
    "beta_ols": beta_ols,
    "beta_2sls": beta_2sls,
    "hausman_stat": hausman_stat,
    "p_value": p_value
}

C:\Users\HP\AppData\Local\Temp\ipykernel_13664\2666195302.py:35: RuntimeWarning: invalid value encountered in sqrt
  hausman_stat = beta_diff / np.sqrt(var_diff)


{'beta_ols': 6.5, 'beta_2sls': 7.2, 'hausman_stat': nan, 'p_value': nan}